#Khai báo thư viện và đọc dữ liệu

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from transformers import TFBertForTokenClassification, BertConfig
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score

In [2]:
# Cài đặt thư viện Kaggle
!pip install kaggle
# Tải lên tệp kaggle.json của bạn chứa thông tin xác thực API
from google.colab import files
files.upload()
# Di chuyển tệp đã tải lên vào thư mục cần thiết
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
# Thiết lập quyền truy cập
#!chmod 600 ~/.kaggle/kaggle.json
# Bây giờ bạn có thể tải tập dữ liệu bằng lệnh API Kaggle
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus
# Giải nén tập dữ liệu
!unzip entity-annotated-corpus.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus
License(s): DbCL-1.0
 95% 25.0M/26.4M [00:02<00:00, 22.2MB/s]
100% 26.4M/26.4M [00:02<00:00, 12.6MB/s]
Archive:  entity-annotated-corpus.zip
  inflating: ner.csv                 
  inflating: ner_dataset.csv         


In [3]:
# Đọc dữ liệu
data = pd.read_csv('ner_dataset.csv', encoding='latin1')
data = data.fillna(method='ffill')
print(data.head(20))

     Sentence #           Word  POS    Tag
0   Sentence: 1      Thousands  NNS      O
1   Sentence: 1             of   IN      O
2   Sentence: 1  demonstrators  NNS      O
3   Sentence: 1           have  VBP      O
4   Sentence: 1        marched  VBN      O
5   Sentence: 1        through   IN      O
6   Sentence: 1         London  NNP  B-geo
7   Sentence: 1             to   TO      O
8   Sentence: 1        protest   VB      O
9   Sentence: 1            the   DT      O
10  Sentence: 1            war   NN      O
11  Sentence: 1             in   IN      O
12  Sentence: 1           Iraq  NNP  B-geo
13  Sentence: 1            and   CC      O
14  Sentence: 1         demand   VB      O
15  Sentence: 1            the   DT      O
16  Sentence: 1     withdrawal   NN      O
17  Sentence: 1             of   IN      O
18  Sentence: 1        British   JJ  B-gpe
19  Sentence: 1         troops  NNS      O


# Xử lý dữ liệu

In [4]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                          s["POS"].values.tolist(),
                                                          s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

getter = SentenceGetter(data)
sentences = getter.sentences

# Chuẩn bị dữ liệu cho RNN và BiLSTM
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)

tags = list(set(data["Tag"].values))
n_tags = len(tags)

In [5]:
# Chuyển đổi dữ liệu
max_len = 75
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Xây dựng các mô hình

In [9]:
# Xây dựng mô hình RNN
model_rnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_words, output_dim=50, input_length=max_len),
    tf.keras.layers.SimpleRNN(units=100, return_sequences=True, recurrent_dropout=0.1),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_tags, activation='softmax'))
])

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 75, 50)            1758900   
                                                                 
 simple_rnn (SimpleRNN)      (None, 75, 100)           15100     
                                                                 
 time_distributed_1 (TimeDi  (None, 75, 17)            1717      
 stributed)                                                      
                                                                 
Total params: 1775717 (6.77 MB)
Trainable params: 1775717 (6.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Xây dựng mô hình BiLSTM
model_bilstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_words, output_dim=50, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=100, return_sequences=True, \
                                                       recurrent_dropout=0.1)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_tags, activation='softmax'))
])

model_bilstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_bilstm.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 75, 50)            1758900   
                                                                 
 bidirectional_2 (Bidirecti  (None, 75, 200)           120800    
 onal)                                                           
                                                                 
 time_distributed_3 (TimeDi  (None, 75, 17)            3417      
 stributed)                                                      
                                                                 
Total params: 1883117 (7.18 MB)
Trainable params: 1883117 (7.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Huấn luyện các mô hình

In [12]:
history_rnn = model_rnn.fit(X_train, np.array(y_train), batch_size=32, \
                            epochs=3, validation_split=0.1)

history_bilstm = model_bilstm.fit(X_train, np.array(y_train), batch_size=32, \
                                  epochs=3, validation_split=0.1)

Epoch 1/3
1214/1214 [==============================] - 164s 129ms/step - loss: 0.1295 - accuracy: 0.9734 - val_loss: 0.0437 - val_accuracy: 0.9875
Epoch 2/3
1214/1214 [==============================] - 120s 99ms/step - loss: 0.0355 - accuracy: 0.9893 - val_loss: 0.0380 - val_accuracy: 0.9886
Epoch 3/3
1214/1214 [==============================] - 117s 96ms/step - loss: 0.0279 - accuracy: 0.9910 - val_loss: 0.0360 - val_accuracy: 0.9889
Epoch 1/3
1214/1214 [==============================] - 580s 472ms/step - loss: 0.1297 - accuracy: 0.9719 - val_loss: 0.0434 - val_accuracy: 0.9877
Epoch 2/3
1214/1214 [==============================] - 563s 464ms/step - loss: 0.0330 - accuracy: 0.9903 - val_loss: 0.0326 - val_accuracy: 0.9902
Epoch 3/3
1214/1214 [==============================] - 558s 459ms/step - loss: 0.0240 - accuracy: 0.9927 - val_loss: 0.0307 - val_accuracy: 0.9906


# Đánh giá mô hình

In [27]:
def evaluate_model(model, X, y):
    loss, accuracy = model.evaluate(X, y)
    print(f"Loss: {loss}, Accuracy: {accuracy}")
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    y_true = np.argmax(y_test, axis=-1).flatten()
    y_pred = y_pred.flatten()
    print("Báo cáo phân loại:")
    print(classification_report(y_true, y_pred, target_names=tags, zero_division='warn'))

In [28]:
# Đánh giá mô hình RNN
print("Đánh giá mô hình RNN:")
evaluate_model(model_rnn, X_test, np.array(y_test))

Đánh giá mô hình RNN:
150/150 [==============================] - 1s 8ms/step - loss: 0.0367 - accuracy: 0.9887
Loss: 0.0366535484790802, Accuracy: 0.9887072443962097
150/150 [==============================] - 1s 7ms/step
Báo cáo phân loại:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       I-gpe       0.93      0.62      0.74        21
       I-org       0.78      0.73      0.76      1725
       B-art       0.50      0.03      0.05        35
       B-nat       0.75      0.21      0.33        28
       B-eve       1.00      0.17      0.29        24
       I-per       0.87      0.84      0.85      1742
       B-geo       0.83      0.87      0.85      3828
       B-org       0.69      0.62      0.65      2072
       I-art       0.00      0.00      0.00        33
       B-tim       0.91      0.82      0.86      2020
       I-nat       0.00      0.00      0.00        10
           O       1.00      1.00      1.00    343475
       I-geo       0.78      0.80      0.79       732
       B-gpe       0.94      0.94      0.94      1556
       I-tim       0.76      0.71      0.74       671
       I-eve       0.00      0.00      0.00        23
       B-per       0.82      0.81      0.82      1705

    accuracy              

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
# Đánh giá mô hình BiLSTM
print("Đánh giá mô hình BiLSTM:")
evaluate_model(model_bilstm, X_test, np.array(y_test))

Đánh giá mô hình BiLSTM:
150/150 [==============================] - 9s 57ms/step - loss: 0.0312 - accuracy: 0.9905
Loss: 0.031210968270897865, Accuracy: 0.9905115365982056
150/150 [==============================] - 16s 107ms/step
Báo cáo phân loại:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       I-gpe       1.00      0.43      0.60        21
       I-org       0.80      0.77      0.78      1725
       B-art       0.00      0.00      0.00        35
       B-nat       0.00      0.00      0.00        28
       B-eve       1.00      0.17      0.29        24
       I-per       0.87      0.82      0.84      1742
       B-geo       0.88      0.88      0.88      3828
       B-org       0.77      0.74      0.75      2072
       I-art       0.00      0.00      0.00        33
       B-tim       0.90      0.89      0.90      2020
       I-nat       0.00      0.00      0.00        10
           O       1.00      1.00      1.00    343475
       I-geo       0.78      0.81      0.80       732
       B-gpe       0.96      0.93      0.95      1556
       I-tim       0.76      0.77      0.76       671
       I-eve       0.00      0.00      0.00        23
       B-per       0.84      0.81      0.83      1705

    accuracy              

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Dự đoán nhãn

In [30]:
idx2tag = {i: t for t, i in tag2idx.items()}

def predict_and_print(model, sentence):
    sentence_transformed = pad_sequences([[word2idx.get(w, word2idx["ENDPAD"]) for w in sentence.split()]], maxlen=max_len, padding="post", value=n_words-1)
    pred = model.predict(sentence_transformed)
    pred_tags = np.argmax(pred, axis=-1)
    words_tags = [(word, idx2tag[pred_tags[0][i]]) if pred_tags[0][i] < len(tag2idx) else (word, 'O') for i, word in enumerate(sentence.split())]
    # In kết quả dự đoán
    print("Kết quả dự đoán:")
    for word, tag in words_tags:
        print(f"{word}: {tag}")

In [40]:
# Dự đoán và in kết quả sử dụng mô hình RNN
print("\nDự đoán và in kết quả sử dụng mô hình BiLSTM:")
predict_and_print(model_rnn, "John Lee is the chief of CBSE, Americans suffered from H5N1 virus in 2002")


Dự đoán và in kết quả sử dụng mô hình BiLSTM:
1/1 [==============================] - 0s 37ms/step
Kết quả dự đoán:
John: B-per
Lee: I-per
is: O
the: O
chief: O
of: O
CBSE,: O
Americans: B-gpe
suffered: O
from: O
H5N1: O
virus: O
in: O
2002: B-tim


In [39]:
# Dự đoán và in kết quả sử dụng mô hình BiLSTM
print("\nDự đoán và in kết quả sử dụng mô hình BiLSTM:")
predict_and_print(model_bilstm, "John Lee is the chief of CBSE, Americans suffered from H5N1 virus in 2002")


Dự đoán và in kết quả sử dụng mô hình BiLSTM:
1/1 [==============================] - 0s 55ms/step
Kết quả dự đoán:
John: B-per
Lee: I-per
is: O
the: O
chief: O
of: O
CBSE,: O
Americans: B-gpe
suffered: O
from: O
H5N1: O
virus: O
in: O
2002: B-tim
